# Distributed PyTorch Operations with nbdistributed

This notebook demonstrates distributed all_gather operations across multiple devices using nbdistributed package.

## Setup Overview
- **Coordinator (Mac)**: Runs this notebook and coordinates operations
- **Worker (Windows RTX 2080)**: Provides GPU computation power

## Requirements
- Both machines must be on the same network
- Windows machine should have CUDA-enabled PyTorch installed
- nbdistributed package installed on both machines


In [ ]:
# Install required packages
%pip install nbdistributed torch torchvision


In [ ]:
import torch
import torch.distributed as dist
from nbdistributed import make_distributed
import os
import socket

# Check if CUDA is available on this machine (Mac)
print(f"CUDA available on Mac: {torch.cuda.is_available()}")
print(f"Mac will run as CPU coordinator (Rank 0)")

# Check for Mac's Metal GPU support (optional)
if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    print("Mac Metal Performance Shaders (MPS) available for local operations")
    mac_device = "mps"
else:
    print("Using CPU for Mac operations")
    mac_device = "cpu"

print(f"Mac device: {mac_device}")

# Get local IP address for network setup
hostname = socket.gethostname()
local_ip = socket.gethostbyname(hostname)
print(f"Mac IP (Master): {local_ip}")
print(f"Hostname: {hostname}")
print("\nSetup: Mac (CPU/MPS coordinator) + Windows (CUDA GPU worker)")


## Configure Distributed Setup

**Step 1**: Note down your Mac's IP address from above. You'll need this for the Windows machine.

**Step 2**: Configure the distributed environment. Update the `WINDOWS_IP` with your Windows laptop's IP address.


In [ ]:
# Distributed configuration
MASTER_ADDR = local_ip  # This Mac will be the master
MASTER_PORT = "12355"   # Port for communication
WORLD_SIZE = 2          # Total number of processes (Mac + Windows)
RANK = 0               # This machine's rank (master = 0)

# TODO: Update this with your Windows laptop's IP
WINDOWS_IP = "192.168.1.XXX"  # Replace with actual Windows IP

print(f"Master Address (this Mac): {MASTER_ADDR}")
print(f"Master Port: {MASTER_PORT}")
print(f"World Size: {WORLD_SIZE}")
print(f"This machine's rank: {RANK}")
print(f"Windows machine IP: {WINDOWS_IP}")

# Set environment variables for distributed training
os.environ['MASTER_ADDR'] = MASTER_ADDR
os.environ['MASTER_PORT'] = MASTER_PORT
os.environ['WORLD_SIZE'] = str(WORLD_SIZE)
os.environ['RANK'] = str(RANK)


In [ ]:
# Initialize distributed process group
@make_distributed
def initialize_distributed():
    """Initialize the distributed environment"""
    # Use 'gloo' backend for CPU-GPU mixed setup (Mac CPU + Windows GPU)
    # 'gloo' supports heterogeneous setups better than 'nccl'
    backend = 'gloo'
    
    print(f"Initializing distributed training with backend: {backend}")
    print(f"Backend 'gloo' chosen for Mac (CPU) + Windows (GPU) setup")
    print(f"Rank: {dist.get_rank()}, World size: {dist.get_world_size()}")
    
    rank = dist.get_rank()
    if rank == 0:
        print("This is the Mac coordinator (Rank 0)")
    else:
        print(f"This is worker rank {rank}")
    
    return backend

# This decorator will handle the distributed setup
backend = initialize_distributed()


## All-Gather Operations Test

Now let's implement the distributed all_gather operations similar to your original code, but adapted for distributed execution.


In [ ]:
@make_distributed
def test_all_gather_basic():
    """Test basic all_gather operation across distributed nodes"""
    
    rank = dist.get_rank()
    world_size = dist.get_world_size()
    
    # Device allocation based on rank and machine type
    if rank == 0:
        # Mac coordinator - use CPU or MPS if available
        device = torch.device(mac_device)  # "cpu" or "mps"
        print(f"Rank {rank} (Mac): Running on device {device}")
    else:
        # Windows worker - should use CUDA
        device = torch.device(f"cuda:0")  # Windows RTX 2080
        print(f"Rank {rank} (Windows): Running on device {device}")
    
    # Create tensors with different data for each rank
    tensor_list = [torch.zeros(3, dtype=torch.int64).to(device) for _ in range(world_size)]
    
    # Create input tensor with rank-specific data
    input_tensor = torch.tensor([rank*100, rank*100+10, rank*100+20], dtype=torch.int64).to(device)
    
    print(f"Rank {rank}: Before all_gather - input: {input_tensor}")
    print(f"Rank {rank}: Before all_gather - tensor_list: {tensor_list}")
    
    # Perform all_gather
    dist.all_gather(tensor_list, input_tensor)
    
    print(f"Rank {rank}: After all_gather - tensor_list: {tensor_list}")
    
    # Synchronize based on device type
    if rank == 0 and mac_device == "mps":
        torch.mps.synchronize()
    elif rank != 0:  # Windows with CUDA
        torch.cuda.synchronize()
    
    return tensor_list

# Run the basic all_gather test
result = test_all_gather_basic()


In [ ]:
@make_distributed  
def test_all_gather_advanced():
    """Test more complex all_gather operations with timing"""
    import time
    
    rank = dist.get_rank()
    world_size = dist.get_world_size()
    
    # Device allocation based on rank and machine type
    if rank == 0:
        # Mac coordinator
        device = torch.device(mac_device)
        machine_type = "Mac"
    else:
        # Windows worker with RTX 2080
        device = torch.device(f"cuda:0")
        machine_type = "Windows RTX 2080"
    
    print(f"\n--- Advanced All-Gather Test ---")
    print(f"Rank {rank} ({machine_type}): Running on device {device}")
    
    # Test with larger tensors
    tensor_size = (2, 3)  # 2x3 tensor
    tensor_list = [torch.zeros(tensor_size, dtype=torch.int64).to(device) for _ in range(world_size)]
    
    # Create input tensor with more complex data
    input_tensor = torch.tensor([[rank*10, rank*10+1, rank*10+2], 
                                [rank*10+3, rank*10+4, rank*10+5]], 
                               dtype=torch.int64).to(device)
    
    print(f"Rank {rank}: Input tensor shape: {input_tensor.shape}")
    print(f"Rank {rank}: Input tensor:\n{input_tensor}")
    
    # Time the operation
    start_time = time.time()
    
    # Perform all_gather
    dist.all_gather(tensor_list, input_tensor)
    
    # Synchronize based on device type
    if rank == 0 and mac_device == "mps":
        torch.mps.synchronize()
    elif rank != 0:  # Windows with CUDA
        torch.cuda.synchronize()
    
    end_time = time.time()
    
    print(f"Rank {rank}: All-gather completed in {end_time - start_time:.4f} seconds")
    print(f"Rank {rank}: Gathered tensors from all ranks:")
    for i, tensor in enumerate(tensor_list):
        machine = "Mac (CPU/MPS)" if i == 0 else f"Windows RTX 2080"
        print(f"  From rank {i} ({machine}):\n{tensor}")
    
    return tensor_list, end_time - start_time

# Run the advanced all_gather test
result_advanced, timing = test_all_gather_advanced()


## Windows RTX 2080 Setup Instructions

**On your Windows laptop with RTX 2080, create this Python script (`worker.py`):**

```python
import torch
import torch.distributed as dist
import os

# Configuration - UPDATE THESE VALUES
MASTER_ADDR = "YOUR_MAC_IP_HERE"  # Replace with your Mac's IP from above
MASTER_PORT = "12355"
WORLD_SIZE = 2
RANK = 1  # Windows machine is rank 1

# Set environment variables
os.environ['MASTER_ADDR'] = MASTER_ADDR
os.environ['MASTER_PORT'] = MASTER_PORT
os.environ['WORLD_SIZE'] = str(WORLD_SIZE)
os.environ['RANK'] = str(RANK)

# Initialize distributed process group
dist.init_process_group(backend='gloo')

print(f"Windows worker initialized with CUDA: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(0)}")
print(f"Rank: {dist.get_rank()}, World size: {dist.get_world_size()}")

# Keep the worker running and ready for operations
try:
    print("Windows worker ready for distributed operations...")
    # This will keep the process alive for distributed operations
    dist.barrier()  # Wait for coordination
    print("Distributed operations completed!")
finally:
    dist.destroy_process_group()
```

**Steps to run:**
1. Install PyTorch with CUDA support on Windows: `pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118`
2. Update `MASTER_ADDR` in the script with your Mac's IP
3. Run: `python worker.py`
4. Then run the cells in this notebook on your Mac

**Network requirements:**
- Both machines on same network
- Port 12355 open for communication
- Firewall may need configuration
